# Summary

Trying to figure out how ConversationManager will work. Having a hard time planning this out so I'm thinking it may be best to try one approach to building it, see what issues arise, and then it will be easier to fix them. Start by trying to subclass PromptManager.

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path

from jabberwocky.config import C
from jabberwocky.external_data import wiki_data
from jabberwocky.openai_utils import load_prompt, load_openai_api_key, \
    PromptManager, print_response
from htools import *

In [3]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [4]:
@auto_repr
class ConversationManager(PromptManager):
    
    def __init__(self, verbose=True, log_dir='data/logs'):
        super().__init__('conversation', verbose=verbose, log_dir=log_dir)
        
    def query(self, text, debug=False, extra_kwargs=None, **kwargs):
        # TODO: reconsider later but trying to simplify things for now.
        assert not extra_kwargs or \
            all(arg not in extra_kwargs for arg in ('stream', 'return_full'))
        assert all (arg not in kwargs for arg in ('stream', 'return_full'))
        
        prompt, resp = super().query('conversation', text, debug=debug, 
                                     extra_kwargs=extra_kwargs, **kwargs)
        return prompt, resp

In [5]:
conv = ConversationManager()
conv

conversation: Your message must start with 'Hi {name}.'. Might want to try tweaking frequency penalty.
-------------------------------------------------------------------------------



ConversationManager(verbose=True, log_dir='data/logs')

In [6]:
conv.prompts

{'conversation': {'engine_i': 3,
  'frequency_penalty': 0.1,
  'max_tokens': 250,
  'prompt': 'This is a conversation with {name}. {summary}\n\nMe: {message}\n\n{name}:',
  'stop': ['Me:', 'This is a conversation with'],
  'temperature': 0.5}}

In [7]:
txt = 'Hi Michael Jordan. Who are some athletes from other sports who you ' \
      'most respect, and why?'
prompt, resp = conv.query(txt)

Writing data to data/logs/query_kwargs.json.


In [10]:
print_response(prompt, resp)

This is a conversation with Michael Jordan. Michael Jeffrey Jordan (born February 17, 1963), also known by his initials MJ, is an American businessman and former professional basketball player. He is the principal owner and chairman of the Charlotte Hornets of the National Basketball Association (NBA) and of 23XI Racing in the NASCAR Cup Series. He played 15 seasons in the NBA, winning six championships with the Chicago Bulls. His biography on the official NBA website states: "By acclamation, Michael Jordan is the greatest basketball player of all time." He was integral in helping to popularize the NBA around the world in the 1980s and 1990s, becoming a global cultural icon in the process.Jordan played college basketball for three seasons under coach Dean Smith with the North Carolina Tar Heels. As a freshman, he was a member of the Tar Heels' national championship team in 1982. Jordan joined the Bulls in 1984 as the third overall draft pick, and quickly emerged as a league star, enter

In [14]:
conv.kwargs('conversation')

{'engine_i': 3,
 'frequency_penalty': 0.1,
 'logprobs': None,
 'max_tokens': 250,
 'mock': False,
 'mock_func': None,
 'mock_mode': 'raise',
 'return_full': False,
 'stop': ['Me:', 'This is a conversation with'],
 'stream': False,
 'strip_output': True,
 'temperature': 0.5}

In [13]:
# Minor Issue: kwargs method expects 'task' param but I'd like that to be 
# automatic. But if we create some sort of partial method there then 
# PromptManager.query() will break.
inspect.signature(conv.kwargs)

<Signature (task, fully_resolved=True, return_prompt=False, extra_kwargs=None, **kwargs)>

First task  
- construct person-specific prompt  
base_prompt = "This is a conversation with {Michael Jordan}. {wiki bio}"

Second task  
- query w/ user text  
GET gpt3_response  
UPDATE running_prompt := base_prompt + 'Me: {user_input}' + gpt3_response  
RETURN running_prompt

In [32]:
class ConversationManager:
    img_exts = {'.jpg', '.jpeg', '.png'}
    
    def __init__(self, verbose=True, data_dir='./data'):
        self.verbose = verbose
        self.data_dir = Path(data_dir)
        self.log_dir = self.data_dir/'logs'
        self.persona_dir = self.data_dir/'conversation_personas'
        
        self._kwargs = load_prompt('conv_proto')
        self._base_prompt = self._kwargs.pop('prompt')
        self.name2summary, self.name2img_path = self.load_personas()
        self.name2base = {k: self._base_prompt.format(name=k, summary=v)
                          for k, v in self.name2summary.items()}

    def load_personas(self):
        name2summary = {}
        name2img_path = {}
        for path in self.persona_dir.iterdir():
            if not path.is_dir(): continue
            name2summary[path.stem] = load(path/'summary.txt')
            name2img_path[path.stem] = [p for p in path.iterdir() 
                                        if p.suffix in self.img_exts][0]
        return name2summary, name2img_path
        
    def load_persona(self, name, download_if_necessary=False):
        try:
            self.name2base[self.process_name(name)]
        except KeyError as e:
            if not download_if_necessary:
                raise e
        return self.add_persona(name, return_data=True)
        
    def add_persona(self, name, return_data=False):
        processed_name = self.process_name(name)
        summary, _, img_path = wiki_data(
            name, img_dir=self.persona_dir/processed_name, fname='profile'
        )
        save(summary, self.persona_dir/processed_name/'summary.txt')
        self.name2summary[processed_name] = summary
        self.name2img_path[processed_name] = img_path
        self.name2base[processed_name] = self._base_prompt.format(
            name=processed_name, summary=summary
        )
        if return_data: return summary, img_path
        
    def process_name(self, name):
        return name.lower().replace(' ', '_')
    
    def __contains__(self, name):
        return self.process_name(name) in self.name2base

In [33]:
conv = ConversationManager()
conv

conv_proto: Your message must start with 'Hi {name}.'. Might want to try tweaking frequency penalty.
-------------------------------------------------------------------------------



In [34]:
conv.load_persona('Bill Gates', download_if_necessary=True)

Writing data to data/conversation_personas/bill_gates/summary.txt.


('William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, author, and philanthropist. He is the co-founder of Microsoft Corporation. During his career at Microsoft, Gates held the positions of chairman, chief executive officer (CEO), president and chief software architect, while also being the largest individual shareholder until May 2014. He is considered one of the best known entrepreneurs of the microcomputer revolution of the 1970s and 1980s.',
 'data/conversation_personas/bill_gates/profile.jpg')